In [113]:
from pyvi import ViTokenizer, ViPosTagger
def remove_pattern(row_content):
    
    return re.sub('[\d]+', '', row_content)
def tokenize(row_content):
    return ViTokenizer.tokenize(row_content)

In [150]:
import pandas
import re

TextFileReader = pd.read_csv('final_tagged_reviews.csv', chunksize=1000, sep=',')  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)

In [159]:
df["Content"] = df["Content"].apply(remove_pattern)
df["Content"]= df["Content"].apply(tokenize)
# df["Content"] = df["Content"].apply(remove_pattern)
df = df.dropna()
df.describe()

,Sentiment
count,20782.000000
mean,0.568810
std,0.495255
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [161]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
reviews = []
for review in df["Content"].values:
    reviews.append(review)
# print(reviews)
stop_words = set(['để', 'thì', 'là','đi','bộ','ai','bên', 'anh', 'bài', 'ba', 'bao', 'mà', 'của', 'trong', 'mình', 'gì', 'những', 'lại', 'và', 'amp', 'quot', 'sẽ', 'phút', 'gì_đó', 'các'])
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words, min_df=2, max_df=0.5, max_features=10000)
X = vectorizer.fit_transform(reviews)
# print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())
print(X.shape)

['about', 'advise', 'again', 'ah', 'all', 'also', 'am', 'an', 'an can', 'an ngon', 'an sang', 'an sáng', 'an tuong', 'an uong', 'an_bàng', 'an_ninh', 'an_toàn', 'an_tâm', 'and', 'and friendly', 'and helpful', 'and the', 'angkor', 'anh_chị', 'anh_chị chủ', 'anh_em', 'ao', 'are', 'are very', 'around', 'as', 'asked', 'at', 'atk', 'available', 'back', 'bacon', 'bad', 'bagan', 'baht', 'bai', 'bai bien', 'bai tam', 'balcony', 'bali', 'ban', 'ban be', 'ban_công', 'ban_công có', 'ban_công nhìn', 'ban_công rất', 'ban_công thoáng', 'ban_công view', 'ban_công đẹp', 'ban_ngày', 'ban_tặng', 'ban_đêm', 'ban_đầu', 'bang', 'bangkok', 'banh', 'banh mi', 'bao_bọc', 'bao_giờ', 'bao_giờ quay', 'bao_giờ đến', 'bao_gồm', 'bao_gồm cả', 'bao_gồm vat', 'bao_gồm ăn', 'bao_la', 'bao_nhiêu', 'bao_phủ', 'bar', 'bat', 'bat tien', 'bath', 'bathroom', 'bay', 'bayon', 'bbq', 'be', 'be boi', 'beach', 'beautiful', 'because', 'bed', 'beepub', 'ben', 'ben ngoai', 'bep', 'best', 'better', 'bi', 'bi hu', 'bi mat', 'bia', 'b

In [3]:
def preprocessing(content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs)
    return X_tfidf

In [164]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib
import pickle

import pandas
import re

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X, df['Sentiment'], test_size=0.1, random_state=69)  
# X_train_tfidf.toarray()
y_train_tfidf
# clf = GaussianNB().fit(X_train_tfidf.toarray(), y_train_tfidf)
# filename = 'finalized_model.sav'
# pickle.dump(clf, open(filename, 'wb'))

1858     1.0
15894    0.0
8644     1.0
533      1.0
11983    0.0
9824     1.0
13013    0.0
4176     1.0
20174    0.0
3466     1.0
20662    0.0
23737    1.0
5113     1.0
8769     1.0
17186    0.0
10056    1.0
9508     1.0
21152    0.0
9802     1.0
5986     1.0
19752    0.0
10250    1.0
11537    1.0
4383     1.0
13418    0.0
2616     1.0
4957     1.0
6091     1.0
15129    0.0
6214     1.0
        ... 
21015    0.0
3432     1.0
13748    0.0
17894    0.0
7297     1.0
13366    0.0
5958     1.0
9885     1.0
22578    1.0
18341    0.0
14321    0.0
7379     1.0
14567    0.0
18786    0.0
1037     1.0
268      1.0
8592     1.0
3494     1.0
13110    0.0
3028     1.0
21529    0.0
21209    0.0
18447    0.0
15184    0.0
10391    1.0
4776     1.0
13106    0.0
17548    0.0
11016    1.0
4422     1.0
Name: Sentiment, Length: 18703, dtype: float64

In [28]:
print(X_test_tfidf.shape)
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))
print(loaded_model)

y_pred = clf.predict(X_test_tfidf.toarray())

(21, 2679)
GaussianNB(priors=None, var_smoothing=1e-09)


In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# average_precision = average_precision_score(test['Sentiment'], y_score)
print(f1_score(y_test_tfidf, y_pred, average="macro"))
print(precision_score(y_test_tfidf, y_pred, average="macro"))
print(recall_score(y_test_tfidf, y_pred, average="macro"))    

0.47500000000000003
0.475
0.475


In [30]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_tfidf, y_pred))  

[[ 0  1]
 [ 1 19]]


In [35]:
import numpy as np
print(np.mean(y_pred == y_test_tfidf))  

0.9047619047619048
